In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 目的：スペインの電力価格を予測
# 目的変数：スペインの電力価格(actual_price)
# 評価指標：RME

import numpy as np
import pandas as pd
import os
import pickle
import gc
# 分布確認
!pip install ydata-profiling
from ydata_profiling import ProfileReport
# 可視化
import matplotlib.pyplot as plt
import seaborn as sns
import polars as pl
# 前処理
from sklearn.preprocessing import StandardScaler, MinMaxScaler,LabelEncoder,OneHotEncoder
# モデリング
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from xgboost import XGBClassifier
import lightgbm as lgb
# 日本語表記
!pip install japanize-matplotlib
import japanize_matplotlib
%matplotlib inline
# パラメータ最適化
!pip install optuna
import optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.1/400.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.0/679.0 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.9 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=384c7e57d378eec9fad25f3d8623890be096cac0ce06a43f7ce4fde84fce57ed
  Stored in directory: /root/.cache/pip/wheels/8d/55/1a/19cd535375ed1ede0c996405ebffe34b196d78e2d9545723a2
Successfully built htmlmin
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.0
    Uninstalling scipy-1.16.0:
     

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 40.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=e8af14936b3c8034a60fb3081ab7c406d2f51c1eebd09a46ddd872578d8fc205
  Stored in directory: /root/.cache/pip/wheels/da/a1/71/b8faeb93276fed10edffcca20746f1ef6f8d9e071eee8425fc
Successfully built japanize-matplotlib
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 20.1 MB/s eta 0:00:00


In [3]:
file_path = '/content/drive/MyDrive/Colab Notebooks/signate/smbc/'

df_train = pd.read_csv(file_path + 'train.csv')
df_test = pd.read_csv(file_path + 'test.csv')
df_sample = pd.read_csv(file_path + 'sample_submit.csv',header=None)

In [4]:
print(df_train.shape)
display(df_train.head())
display(df_train.info())

(26280, 92)


,time,generation_biomass,generation_fossil_brown_coal/lignite,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_hydro_pumped_storage_consumption,generation_hydro_run_of_river_and_poundage,generation_hydro_water_reservoir,generation_nuclear,...,seville_wind_deg,seville_rain_1h,seville_rain_3h,seville_snow_3h,seville_clouds_all,seville_weather_id,seville_weather_main,seville_weather_description,seville_weather_icon,price_actual
0,2015-01-02 00:00:00+01:00,447.0,329.0,4844.0,4821.0,162.0,863.0,1051.0,1899.0,7096.0,...,33,0.0,0.0,0.0,0,800,clear,sky is clear,01n,64.02
1,2015-01-02 01:00:00+01:00,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,1658.0,7096.0,...,33,0.0,0.0,0.0,0,800,clear,sky is clear,01n,58.46
2,2015-01-02 02:00:00+01:00,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,1371.0,7099.0,...,44,0.0,0.0,0.0,0,800,clear,sky is clear,01n,54.70
3,2015-01-02 03:00:00+01:00,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,779.0,7098.0,...,44,0.0,0.0,0.0,0,800,clear,sky is clear,01n,54.91
4,2015-01-02 04:00:00+01:00,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,7097.0,...,44,0.0,0.0,0.0,0,800,clear,sky is clear,01n,53.07


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26280 entries, 0 to 26279
Data columns (total 92 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   time                                         26280 non-null  object 
 1   generation_biomass                           26259 non-null  float64
 2   generation_fossil_brown_coal/lignite         26260 non-null  float64
 3   generation_fossil_gas                        26260 non-null  float64
 4   generation_fossil_hard_coal                  26260 non-null  float64
 5   generation_fossil_oil                        26259 non-null  float64
 6   generation_hydro_pumped_storage_consumption  26259 non-null  float64
 7   generation_hydro_run_of_river_and_poundage   26259 non-null  float64
 8   generation_hydro_water_reservoir             26260 non-null  float64
 9   generation_nuclear                           26261 non-null  float64
 10

None

In [5]:
print(df_test.shape)
display(df_test.head())
display(df_test.info())

(8760, 91)


,time,generation_biomass,generation_fossil_brown_coal/lignite,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_hydro_pumped_storage_consumption,generation_hydro_run_of_river_and_poundage,generation_hydro_water_reservoir,generation_nuclear,...,seville_wind_speed,seville_wind_deg,seville_rain_1h,seville_rain_3h,seville_snow_3h,seville_clouds_all,seville_weather_id,seville_weather_main,seville_weather_description,seville_weather_icon
0,2018-01-01 00:00:00+01:00,279.0,0.0,3927.0,895.0,189.0,230.0,1069.0,1893.0,7104.0,...,1,343,0.0,0.0,0.0,0,800,clear,sky is clear,01n
1,2018-01-01 01:00:00+01:00,282.0,0.0,3948.0,878.0,177.0,1269.0,1058.0,1024.0,7101.0,...,1,343,0.0,0.0,0.0,0,800,clear,sky is clear,01n
2,2018-01-01 02:00:00+01:00,283.0,0.0,3791.0,890.0,175.0,2197.0,1052.0,888.0,7100.0,...,1,0,0.0,0.0,0.0,0,800,clear,sky is clear,01n
3,2018-01-01 03:00:00+01:00,280.0,0.0,3671.0,881.0,175.0,2965.0,1032.0,645.0,7101.0,...,1,40,0.0,0.0,0.0,0,800,clear,sky is clear,01n
4,2018-01-01 04:00:00+01:00,286.0,0.0,3460.0,861.0,173.0,2705.0,1001.0,661.0,7101.0,...,1,30,0.0,0.0,0.0,0,800,clear,sky is clear,01n


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 91 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   time                                         8760 non-null   object 
 1   generation_biomass                           8758 non-null   float64
 2   generation_fossil_brown_coal/lignite         8758 non-null   float64
 3   generation_fossil_gas                        8758 non-null   float64
 4   generation_fossil_hard_coal                  8758 non-null   float64
 5   generation_fossil_oil                        8758 non-null   float64
 6   generation_hydro_pumped_storage_consumption  8758 non-null   float64
 7   generation_hydro_run_of_river_and_poundage   8758 non-null   float64
 8   generation_hydro_water_reservoir             8758 non-null   float64
 9   generation_nuclear                           8758 non-null   float64
 10  

None

In [6]:
print(df_sample.shape)
display(df_sample.head())

(8760, 2)


,0,1
0,2018-01-01 00:00:00+01:00,0
1,2018-01-01 01:00:00+01:00,0
2,2018-01-01 02:00:00+01:00,0
3,2018-01-01 03:00:00+01:00,0
4,2018-01-01 04:00:00+01:00,0


In [7]:
# カテゴリカルカラムの抜き出し
categorical_cols = [
    col for col in df_train.select_dtypes(include=['object', 'category']).columns
    if col != "price_actual"
    ]
# 数値カラムの抜き出し
numerical_cols = [
    col for col in df_train.select_dtypes(include=['int64', 'float64']).columns
    ]
print('*'*20,'実行結果確認','*'*20,)
print(f'カテゴリ{categorical_cols}')
print(f'数値{numerical_cols}')


******************** 実行結果確認 ********************
カテゴリ['time', 'valencia_weather_main', 'valencia_weather_description', 'valencia_weather_icon', 'madrid_weather_main', 'madrid_weather_description', 'madrid_weather_icon', 'bilbao_weather_main', 'bilbao_weather_description', 'bilbao_weather_icon', 'barcelona_weather_main', 'barcelona_weather_description', 'barcelona_weather_icon', 'seville_weather_main', 'seville_weather_description', 'seville_weather_icon']
数値['generation_biomass', 'generation_fossil_brown_coal/lignite', 'generation_fossil_gas', 'generation_fossil_hard_coal', 'generation_fossil_oil', 'generation_hydro_pumped_storage_consumption', 'generation_hydro_run_of_river_and_poundage', 'generation_hydro_water_reservoir', 'generation_nuclear', 'generation_other', 'generation_other_renewable', 'generation_solar', 'generation_waste', 'generation_wind_onshore', 'total_load_actual', 'valencia_temp', 'valencia_temp_min', 'valencia_temp_max', 'valencia_pressure', 'valencia_humidity', 'val

In [8]:
# 訓練データ、テストデータ作成（＝目的変数分離）
x_train = df_train[['generation_biomass','generation_fossil_brown_coal/lignite']]
y_train = df_train['price_actual']
x_test = df_test[['generation_biomass','generation_fossil_brown_coal/lignite']]

In [9]:
from sklearn.metrics import mean_squared_error
# 交差検証
FOLDS = 5
# skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)
kf = KFold(n_splits=FOLDS, shuffle=True, random_state=42)

# oof:out of fold/テスト結果格納/スコアリスト
oof = np.zeros(len(y_train))
preds = np.zeros(len(x_test))
rmse_list = []

# パラメータ
params = {
    'boosting_type':'gbdt',
    'objective':'regression',
    'metric':'rmse',
    'learning_rate':0.1,
    'num_leaves':16,
    'n_estimators':10000,
    'random_state':123,
    'importance_type':'gain'
}

# 交差検証分繰り返す
for i, (train_idx, valid_idx) in enumerate(kf.split(x_train,y_train)):
  print('#' * 15, i+1, '#' * 15)
  x_tr, y_tr = x_train.iloc[train_idx], y_train.iloc[train_idx]
  x_va, y_va = x_train.iloc[valid_idx], y_train.iloc[valid_idx]
  # モデルを訓練する
  lgb_model = lgb.LGBMRegressor(**params)
  lgb_model.fit(
      x_tr, y_tr,
      eval_set=[(x_va, y_va)],
      callbacks=[
        lgb.early_stopping(stopping_rounds=50), #100に変えてもよいかも
        lgb.log_evaluation(period=10) # verboseの設定
        ]
    )
  # 検証用データに対する予測を書き込む
  y_pred = lgb_model.predict(x_va)
  oof[valid_idx] = y_pred

  # テストデータ予測を累積
  preds += lgb_model.predict(x_test)

  # RMSE を計算（squared=False で平方根を取る）
  mse = mean_squared_error(y_va, y_pred)
  rmse = np.sqrt(mse)
  rmse_list.append(rmse)
  print(f"RMSE: {rmse:.4f}")


# 最後にテスト予測を平均化
preds /= FOLDS

print(f'平均 RMSE: {np.mean(rmse_list):.4f}')
print(f'OOF array shape: {oof.shape}')
print(f'Preds array shape: {preds.shape}')

############### 1 ###############
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000962 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 507
[LightGBM] [Info] Number of data points in the train set: 21024, number of used features: 2
[LightGBM] [Info] Start training from score 56.089827
Training until validation scores don't improve for 50 rounds
[10]	valid_0's rmse: 13.0708
[20]	valid_0's rmse: 12.8518
[30]	valid_0's rmse: 12.7925
[40]	valid_0's rmse: 12.7646
[50]	valid_0's rmse: 12.745
[60]	valid_0's rmse: 12.7325
[70]	valid_0's rmse: 12.7217
[80]	valid_0's rmse: 12.7157
[90]	valid_0's rmse: 12.7139
[100]	valid_0's rmse: 12.7119
[110]	valid_0's rmse: 12.7083
[120]	valid_0's rmse: 12.7043
[130]	valid_0's rmse: 12.7016
[140]	valid_0's rmse: 12.701
[150]	valid_0's rmse: 12.6996
[160]	valid_0's rmse: 12.6979
[170]	valid_0's rmse: 12.6995
[180]	valid_0's rmse: 12.7015
[190]	valid_0's rmse: 12.704
[200]	

In [10]:
def train_model(input_x,input_y,input_test,params,folds):
  '''XGBoostモデルを回す学習用の関数

  arg:
    input_x:trainデータの説明変数
    input_y:trainデータの目的変数
    input_test:testデータ（できればここはなくしたいが現実力では思いつかず）
    params:XGBoostのパラメータ。辞書型を想定
    folds:交差検証、n_splits用の数字

  return:
    oof:訓練データ全体の各行について、「自分が検証データになったときの予測確率」を保持した配列。
    pred_prob:交差検証の各 fold で得られたテストデータの予測確率を累積し、最後に fold 数で割って平均したもの。
    map3_scores_list:MAP@3評価のスコア結果

  '''

  oof = np.zeros(shape = (len(input_x), input_y.nunique()))
  pred_prob = np.zeros(shape = (len(input_test),input_y.nunique()))
  map3_scores_list = []
  skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)

  for i, (train_idx, valid_idx) in enumerate(skf.split(input_x,input_y)):
    print('#' * 15, i+1, '#' * 15)
    x_tr, y_tr = input_x.iloc[train_idx], input_y.iloc[train_idx]
    x_va, y_va = input_x.iloc[valid_idx], input_y.iloc[valid_idx]
    xgb_model = XGBClassifier(**params)
    # モデルを訓練する
    xgb_model.fit(x_tr,y_tr, eval_set=[(x_va,y_va)], verbose = 100)
    # 検証用データに対する確率予測を書き込む
    oof[valid_idx] = xgb_model.predict_proba(x_va)
    pred_prob += xgb_model.predict_proba(input_test)

    top_3_preds = np.argsort(oof[valid_idx], axis=1)[:, -3:][:, ::-1]
    actual = [[label] for label in y_va]
    map3_score = mapk(actual, top_3_preds)
    map3_scores_list.append(map3_score)
    print(f"✅ FOLD {i+1}: MAP@3 Score: {map3_score:.5f}")
  return oof,pred_prob,map3_scores_list

In [11]:
from datetime import date

today = date.today()

df_sub = pd.read_csv(file_path + 'sample_submit.csv',header=None)

submission = pd.DataFrame({
    'time':df_sub.iloc[:, 0] ,
    'pred': preds
})
submission.to_csv(f'submission_{today}.csv', index=False, header=False)
print(f'✅ Submission file saved as submission_{today}.csv')

✅ Submission file saved as submission_2025-07-30.csv
